In [1]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))


from utils.augmentations import get_default_transform
from utils import creating_dataset


# Dataset loading & inspection
This notebook shows how to load the datasets used in this challenge and provides some basic statistics about them.

Note that the data includes EMG signal from 8 electrodes in the EMG armband, the predicted variables are the angle of 20 joints in the hand. Inputs are sampled at 200Hz, but the  outputs are intended to be at a subsampled rate of 25Hz (more on that in other notebooks).

Data was acquired in "healthy" and "amputant" subjects (i.e. with limb loss) using the EMG armband in either the left or right arm. 
Your model's predictions will be evaluated on data from one of the two amputant subjects (fedya), but you can use any and all provided data for training your model. For simplicity, the data has been divided in `training` and `test` subsets (not all subjects have `test` data) to evaluate your model's performance during training. The final submission data is held separate and only used when preparing the submission.csv file (see 04_submit_predictions.ipynb).

### Load data
Start by defining a variable to keep track of where the data is saved on your computer and a set of parameters for selecting which data to load. 

In [2]:
DATA_PATH = r"C:\Users\nodos\Desktop\ДИПЛОМ\Аня диплом\dataset_v2_blocks\dataset_v2_blocks"

You can decide which data to load (e.g. from both `health` and `amputant` subjects). 
The `test_dataset_list` specifies which subset of the data should be used as test set, and it shouldn't change. You can, however, play around with training your model on different subsets of the available training data. 

You can also define a set of `transform` functions to apply to the data before feeding it to the model, or load the default ones.

In [3]:
data_paths = dict(
    datasets=[DATA_PATH],
    hand_type = ['left', 'right'], # [left, 'right']
    human_type = ['health', 'amputant'], # [amputant, 'health']
    test_dataset_list = ['fedya_tropin_standart_elbow_left']  # don't change this !
)

# define a config object to keep track of data variables
data_config = creating_dataset.DataConfig(**data_paths)

# get transforms
p_transform = 0.1  # probability of applying the transform
transform = get_default_transform(p_transform)

# load the data
train_dataset, test_dataset = creating_dataset.get_datasets(data_config, transform=transform)


Getting val datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Getting train datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True
Number of moves: 70 | Dataset: valery_first_standart_elbow_left
Reorder this dataset valery_first_standart_elbow_left True
Number of moves: 135 | Dataset: alex_kovalev_standart_elbow_left
Reorder this dataset alex_kovalev_standart_elbow_left True
Number of moves: 72 | Dataset: anna_makarova_standart_elbow_left
Reorder this dataset anna_makarova_standart_elbow_left True
Number of moves: 62 | Dataset: artem_snailbox_standart_elbow_left
Reorder this dataset artem_snailbox_standart_elbow_left True
Number of moves: 144 | Dataset: matthew_antonov_standart_elbow_left
Reorder this dataset matthew_antonov_standart_elbow_left True
Number of moves: 144 | Dataset: misha_korobok_standart_elbow_left
Reorder this da

### Inspect the data

`train_dataset` and `test_dataset` are instances of the `torch.utils.data.ConcatDataset` class. 


The following code shows the number of batches in each set as well as the size of input and outputs. 
Note that inputs are of shape `n_channels x batch_size` while the outputs are of shape `n_angles x downsampled_batchs_size` where `downsampled_batch_size = batch_size / 200 * 25` to account for downsampling of predictions. 

In [8]:
print(f"Train dataset size: {len(train_dataset)}, Test dataset size: {len(test_dataset)}")

X, Y = train_dataset[0]
print(f"X shape: {X.shape}, Y shape: {Y.shape}")

Train dataset size: 99990, Test dataset size: 792
X shape: (8, 256), Y shape: (20, 32)


In [16]:
train_dataset[99989]

(array([[ 0.01960784, -0.01960784, -0.01960784, ...,  0.00392157,
         -0.00392157, -0.01176471],
        [-0.00392157, -0.01176471,  0.01176471, ..., -0.01960784,
         -0.01176471, -0.00392157],
        [ 0.00392157,  0.00392157, -0.01176471, ..., -0.02745098,
          0.01176471, -0.01960784],
        ...,
        [ 0.01176471, -0.01176471, -0.01176471, ...,  0.01176471,
         -0.00392157, -0.03529412],
        [ 0.00392157,  0.05882353,  0.06666667, ...,  0.12156863,
          0.05882353, -0.00392157],
        [-0.00392157,  0.01960784,  0.01176471, ...,  0.07450981,
          0.01176471, -0.01960784]], dtype=float32),
 array([[ 0.20623668,  0.24071379,  0.26552832,  0.3069286 ,  0.32817173,
          0.3431184 ,  0.3519675 ,  0.3418238 ,  0.23751955,  0.15299413,
          0.0825957 , -0.267643  , -0.7784495 , -0.84479696, -0.786401  ,
         -0.8499064 , -0.88851607, -0.9086246 , -0.91252273, -0.9102079 ,
         -0.9040388 , -0.8702784 , -0.8096402 , -0.5368706 , -

You can also generate a video of the hand movements.

In [7]:
from utils.hand_visualize import Hand, save_animation
from utils.quats_and_angles import get_quats
import numpy as np


batches = [train_dataset[i] for i in range(100)]
Y = np.concatenate([b[1] for b in batches], axis=1)
quats = get_quats(Y)

hand_gt = Hand(quats)
ani = hand_gt.visualize_all_frames()
save_animation(ani, 'test_vis_1.gif', fps=5,)   # this will save a .gif file